In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
from tqdm.notebook import tqdm
sns.set()
rcParams['figure.figsize'] = (20,10)
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')
from pprint import pprint
import json
from collections import defaultdict
from googletrans import Translator
import time
from datasets import Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModel, AutoConfig, pipeline
from datasets import load_dataset, Dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloom-3b')

In [ ]:
tokenizer._pad_token

In [ ]:
translator = Translator()
def translate_df(df, batch_size=1, translator=translator):
    def translate_text(text, sleep_time=2):
        if text in [''] or text is None:
            return ''
        while True:
            try:
                return translator.translate(text, src="en", dest="vi").text
            except:
                print("Requests error")
                time.sleep(sleep_time)
    
    def clean_text(text):
        return text
    
    data = defaultdict(list)
    for i in tqdm(range(0, len(df), batch_size)):
        batch_df = df.iloc[i : i + batch_size]
        temp_data = defaultdict(list)
        for col in df.columns:
            batch_df[col] = batch_df[col].apply(lambda x: clean_text(x))
            col_vals = ' ## '.join(batch_df[col].values)
            temp_data[col] = [x.strip() for x in translate_text(col_vals).split('##')]

        can_insert_batch = True
        for col in df.columns:
            if len(temp_data[col]) != len(batch_df):
                can_insert_batch = False
                break

        if can_insert_batch:
            for col in df.columns:
                data[col].extend(batch_df[col].values)
                data[f'{col}_translated'].extend(temp_data[col])
            continue
        
        # else loop through each row
        for _, row in batch_df.iterrows():
            for col in df.columns:
                text = clean_text(row[col])
                data[col].append(text)
                data[f'{col}_translated'].append(translate_text(text))
                
    df = pd.DataFrame(data)
    return df.drop(columns=[col for col in df.columns if '_translated' not in col]) 

In [ ]:
a = """Cuộc trò chuyện giữa con người và trợ lý AI.
[|Con người|] {prompt}
[|AI|] {response}
[|Con người|]"""

def get_prompt(row):
    return a.format(prompt=row['prompt'], response=row['response'])

In [ ]:
temp = pd.read_json('../../data/translated/vi_alpaca_reduced.jsonl', lines=True)
temp2 = temp.apply(lambda x: get_prompt(x), axis=1)
temp2 = pd.DataFrame(temp2)
temp2.columns = ['prompt']

In [ ]:
temp3 = pd.read_json('../../data/translated/quora_chat_data_translated.json', orient='records')

In [ ]:
temp3.to_json('../../data/translated/quora_chat_data_translated.json',lines=True,orient='records')
temp2.to_json('../../data/translated/alpaca_chat_cleaned_translated.json',lines=True,orient='records')

## Clean ShareGPT

In [ ]:
share_gpt = pd.read_json('../../data/original/ShareGPT_V3_unfiltered_cleaned_split.json')

In [ ]:
popular_programming_keywords = [
    "python",
    "java",
    "javascript",
    "c#",
    "c++",
    "typescript",
    "ruby",
    "swift",
    "kotlin",
    "php",
    "rust",
    "scala",
    "dart",
    "matlab",
    "objective-c",
    "perl",
    "lua",
    "assert",
    "async",
    "await",
    "def",
    "elif",
    "lambda",
    "nonlocal",
    "function",
    "var",
    "cuda",
    "torch",
    "code",
    "sudo",
    "bash"
]
popular_programming_languages = [
    "python",
    "java",
    "javascript",
    "typescript",
    "kotlin",
    "objective-c",
]
def check_text_not_contain_code(text):
    temp2 = text.lower()
    temp = set(temp2.split())
    for language in popular_programming_keywords:
        if language in temp:
            return False
    for language in popular_programming_languages:
        if language in temp2:
            return False
    return True

temp = share_gpt['conversations'].apply(lambda x: str(x)).apply(check_text_not_contain_code)


In [ ]:
share_gpt_no_code = share_gpt[temp][['conversations']]
share_gpt_no_code = share_gpt_no_code[share_gpt_no_code['conversations'].apply(lambda x: len(x) >= 4)]
share_gpt_no_code.shape

In [ ]:
from numpy import nan


def create_conversation(turns):
    res = "The conversation between human and AI assistant.\n"
    for turn in turns:
        if turn['from'] == 'human':
            res += "[|Human|] " + turn['value'] + "\n"
        elif turn['from'] == 'gpt':
            res += "[|AI|] " + turn['value'] + "\n"
        else:
            return nan
    return res

share_gpt_no_code_conversations = share_gpt_no_code.apply(lambda x: create_conversation(x['conversations']), axis=1)

In [ ]:
print(share_gpt_no_code_conversations.sample(1).iloc[0])

In [ ]:
share_gpt_no_code_conversations.to_json('../../data/original/share_gpt_no_code_conversations.json', orient='records', lines=True)

## FAQ

In [ ]:
path = '../../data/translated/all_faqs.json'
faqs = pd.read_json(path)

In [ ]:
faqs

In [ ]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs.sample(1).iloc[0]))

In [ ]:
temp = faqs['input'].apply(lambda x: not x.endswith('Điều luật liên quan: '))
faqs = faqs[temp]

In [ ]:
faqs

In [ ]:
faqs['output_len'] = faqs['output'].apply(lambda x: len(x.split()))

In [ ]:
faqs['output_len'].hist(bins=100)

In [ ]:
faqs_standard = faqs[faqs['output_len'] < 200]

In [ ]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs_standard.sample(1).iloc[0]))

In [ ]:
template = """{instruction}\n{input}\nCâu trả lời: {output}"""
print(template.format(**faqs.sample(1).iloc[0]))

In [ ]:
faqs_standard

In [ ]:
temp = Dataset.from_json('/Users/phamhoang1408/Desktop/Phase 2 Viettel/main_repo/data/training/alpaca_chat_cleaned_51k_translated.json')
temp.rename_column(temp.column_names[0], 'text')

In [ ]:
ds = Dataset.from_json('../../data/original/gpt4-instruct-similarity-0.8-dataset.json')

In [ ]:
len(ds.shuffle()[0]['response'].split())

In [ ]:
ds

In [ ]:
from datasets import Dataset, concatenate_datasets
from glob import glob

def load_dataset(folder_path, dataset_size=None):
    data = []
    file_paths = glob(folder_path + "/*.jsonl")
    for path in file_paths:
        ds = Dataset.from_json(path)
        if len(ds.column_names) != 1:
            raise ValueError("Dataset must have only one text column")
        ds = ds.rename_column(ds.column_names[0], "text")
        data.append(ds)
    if dataset_size is None:
        return concatenate_datasets(data, axis=0).shuffle()
    return concatenate_datasets(data, axis=0).shuffle().select(range(dataset_size))

In [ ]:
ds = load_dataset('../../data/training_31_7')

In [ ]:
print(ds.shuffle()[0]['text'])

In [ ]:
from glob import glob
from datasets import Dataset, concatenate_datasets
def load_dataset(folder_path, dataset_size=None):
    data = []
    for path in glob(folder_path + "/*.jsonl"):
        ds = Dataset.from_json(path)
        if len(ds.column_names) != 1:
            raise ValueError("Dataset must have only one text column")
        ds = ds.rename_column(ds.column_names[0], "text")
        data.append(ds)
    if dataset_size is None:
        final_ds = concatenate_datasets(data, axis=0).shuffle(seed=42)
    final_ds = (
        concatenate_datasets(data, axis=0).shuffle(seed=42).select(range(dataset_size))
    )
    final_ds = final_ds.filter(lambda x: x["text"] != "" or x["text"] is not None)
    return final_ds

In [ ]:
ds = load_dataset('../../data/training_31_7')

In [ ]:
df = ds.to_pandas()

In [ ]:
def check(x):
    if x == '' or x is None:
        return True
    return False
temp = df['text'].apply(check)

In [ ]:
temp.sum()

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz-3b')

In [ ]:
tokenizer

In [ ]:
tokenizer.tokenize('')

In [3]:
en_tree = []
with open('../../data/original/oasst_ready_trees.json') as f:
    for line in f:
        temp = json.loads(line)
        if temp['prompt']['lang'] in ['en']:
            en_tree.append(temp)

In [4]:
vi_tree = []
with open('../../data/original/oasst_ready_trees.json') as f:
    for line in f:
        temp = json.loads(line)
        if temp['prompt']['lang'] in ['vi']:
            vi_tree.append(temp)

In [5]:
def traverse(root):
    if len(root['replies']) == 0:
        return [[(root['role'], root['text'])]]
    data = []
    for reply in root['replies']:
        data.extend(traverse(reply))
    data = [[(root['role'], root['text'])] + d for d in data]
    return data

In [6]:
en_conversations = []
for tree in en_tree:
    en_conversations.extend(traverse(tree['prompt']))

vi_conversations = []
for tree in vi_tree:
    vi_conversations.extend(traverse(tree['prompt']))

In [7]:
len(en_conversations)

21164

In [8]:
len(vi_conversations)

129

In [15]:
templates = "Cuộc trò chuyện giữa con người và trợ lý AI.\n{conversation}"
def format(conversations):
    data = []
    for conversation in conversations:
        temp = ""
        for turn in conversation:
            if turn[0] == 'prompter':
                temp += f"[|Con người|] {turn[1]}\n"
            else:
                temp += f"[|AI|] {turn[1]}\n"
        data.append(templates.format(conversation=temp))
    return data

In [11]:
en_data = format(en_conversations)

In [16]:
vi_data = format(vi_conversations)

In [33]:
print(en_data[11])

The conversation between human and AI assistant.
[|Human|] Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
[|AI|] Can you write a formal letter to introduce Jeff Bezos to a customer?
[|Human|] I asked about contrastive learning in machine learning which has nothing to do with Jeff Bezos. Contrastive learning is used to increase the performance of vision-based tasks using contrast. I want you to explain this technique to me in a way that anyone without machine learning knowledge would understand.
[|AI|] In simple terms, contrastive learning focuses on teaching an AI the points of similarity between different images (or other media) to indirectly aid it in spotting points of divergence when present. To anthropomorphize the process, a human engaged in contrastive learning and eating hundreds of apples in a week would be better equipped to recognize an orange when presented with one.



In [24]:
en_df = pd.DataFrame(en_data, columns=['text'])
en_df.to_json('../../data/original/en_oasst.json', orient='records', lines=True)